In [1]:
!pip install kaggle

In [2]:
import kaggle
!kaggle datasets download -d snap/amazon-fine-food-reviews

amazon-fine-food-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
from zipfile import ZipFile
file_name = 'amazon-fine-food-reviews.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [4]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [5]:
# using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite')

In [6]:
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3""", con) 

In [7]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

In [8]:
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [10]:
display = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId = "AR5J8UI46CURR"
ORDER BY ProductId
""", con)

display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [11]:
#Sorting data according to ProductId in ascending order
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True,inplace=False, kind='quicksort', na_position='last')

In [12]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [13]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

Observation: It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calculations

In [14]:
display = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id= 44737 OR Id= 64422
ORDER BY ProductID
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [15]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [16]:
#Before starting the next phase of preprocessing lets see the number pf entires
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


Score
1    307061
0     57110
Name: count, dtype: int64

Bag of Words(BoW)

In [17]:
#Bow
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(final['Text'].values)

In [18]:
type(final_counts)

scipy.sparse._csr.csr_matrix

In [19]:
final_counts.get_shape()

(364171, 115281)

Text Preprocessing

Now that we have finshed dedupication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing we do the following in the order below:

Begin by removing the HTML tags
Remove any punctuations or limited set of special characters like , or . or # etc.
Check if the word is made up of english letters and is not alpha- numeric
Check to see if the length of the word is greater than 2( as it was researched that there is no adjective in 2-letters)
Convert the word to lowercase
Remove Stopwords
Finally Snowball Stemming the word(it was obsereved to be better than Porter Stemming)
After which we collect the words used to describe postive and negative reviews

In [20]:
#find sentences containing HTML tags

i = 0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i += 1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [21]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import nltk

nltk.download('stopwords')  # Download stopwords

stopwords = set(stopwords.words('english'))  # Assign downloaded stopwords

def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]', r'', sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r'', cleaned)
    return cleaned

print(stopwords)
print('**************************')

sno = PorterStemmer()
print(sno.stem('tasty'))


{'not', 'on', 'should', "hasn't", 'which', 'your', 'this', 'each', 'her', 'doesn', 'own', 'themselves', "mightn't", 'while', 'weren', 'a', "hadn't", 'or', 'were', 'don', 'now', 've', 'hasn', 'about', 'before', 'over', 'and', 'into', 'has', "should've", 'between', 'needn', 'some', 'they', 'we', 'all', "won't", 'm', 'from', 'hadn', 'was', 'its', 'herself', 'yourselves', "isn't", 'few', "shouldn't", 'mustn', 'myself', 'i', 'to', 'below', "weren't", 'do', 'where', 'will', "aren't", 'no', 'until', 'other', 'same', 'with', 'shouldn', 'both', 'o', "doesn't", 'by', 'too', 'won', 'yourself', 'shan', 'you', 'am', "couldn't", "wasn't", 'any', 'as', 'further', 'didn', 'these', 'have', 'she', 'what', 's', 'is', 'them', 'd', 're', 'when', 'his', 'ourselves', "shan't", 'yours', 'there', 'once', "she's", 'in', 'ma', 'aren', 'of', 'ours', 'through', 'only', 'ain', 'here', 'who', 'does', 'down', 'more', "mustn't", 'isn', "needn't", 'most', 'y', 'be', 'it', 'again', 'such', 'out', 'mightn', 'did', 'than'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/roushankumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
i = 0
str1 = ' '
final_string = []
all_positive_words = []
all_negative_words = []
s = ''
for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(((cleaned_words.isalpha()) & (len(cleaned_words) > 2))):
                if(cleaned_words.lower() not in stopwords):
                    s = (sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if (final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence)
    final_string.append(str1)
    i += 1

In [23]:
print(all_positive_words)

[]


In [24]:
final['CleanedText'] = final_string

In [25]:
final.head(3)

#store final table into an SQlLite table for future
conn = sqlite3.connect('final.sqllite')
c = conn.cursor
conn.text_factory = str
final.to_sql('Reviews', conn,schema=None,if_exists='replace')

364171

In [26]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,b'great littl book read nice rhythm well good ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,b'book poetri month year goe month cute littl ...


Bi-Grams and n-Grams

Goal

Now that we have our list of words describing positive and negative reviews lets analyse them.

We begin analysis by getting the frequency distibution of the words as shown below

In [27]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)
print("Most Common Positive Words : ", freq_dist_positive.most_common(20))
print("Most Common Negative Words : ", freq_dist_negative.most_common(20))

Most Common Positive Words :  []
Most Common Negative Words :  []


Observation:- From the above it can be seen that the most common positive and the negative words overlap for eg. 'like' could be used as 'not like' etc. So, it is a good idea to consider pairs of consequent words(bi-grams) or q sequence of n consecutive words (n-grams)

In [28]:
#bi-gram, tri-gram and n-gram

#removing stop words like 'not' should be avoided before building n-grams
count_vect = CountVectorizer(ngram_range=(1,2))
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [29]:
final_bigram_counts.get_shape()

(364171, 2910192)

<b>TF-IDF</b>

In [30]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [31]:
final_tf_idf.get_shape()

(364171, 2910192)

In [32]:
features = tf_idf_vect.get_feature_names_out()
len(features)

2910192

In [33]:
features[100000:100010]

array(['ales until', 'ales ve', 'ales would', 'ales you', 'alessandra',
       'alessandra ambrosia', 'alessi', 'alessi added', 'alessi also',
       'alessi and'], dtype=object)

In [34]:
#convert a row in saprsematirx to a numpy array
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [35]:
#source https://buhrmann.github.io/tfidf-analysis.html

def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [36]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


In [ ]:
import kaggle
!kaggle datasets download -d leadbest/googlenewsvectorsnegative30

In [37]:
file_names = 'archive.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [39]:
#Using Google News W2V

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, encoding='latin1')

In [40]:
model['computer']

array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -

In [41]:
model.similarity('woman','man')

0.76640123

In [42]:
model.most_similar('woman')

[('man', 0.7664012908935547),
 ('girl', 0.7494640946388245),
 ('teenage_girl', 0.7336829900741577),
 ('teenager', 0.631708562374115),
 ('lady', 0.6288785934448242),
 ('teenaged_girl', 0.6141783595085144),
 ('mother', 0.6076306104660034),
 ('policewoman', 0.6069462895393372),
 ('boy', 0.5975909233093262),
 ('Woman', 0.5770983099937439)]

In [44]:
model.most_similar('tasty')

[('delicious', 0.8730388283729553),
 ('scrumptious', 0.8007041215896606),
 ('yummy', 0.7856924533843994),
 ('flavorful', 0.7420163750648499),
 ('delectable', 0.7385422587394714),
 ('juicy_flavorful', 0.7114803791046143),
 ('appetizing', 0.7017217874526978),
 ('crunchy_salty', 0.701229989528656),
 ('flavourful', 0.6912214159965515),
 ('flavoursome', 0.6857702732086182)]

In [45]:
model.similarity('tasty','tast')

0.44035056

In [46]:
#Train own Word2Vec model using own text corpus

import gensim
i = 0
list_of_sent = []
for sent in final['Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [47]:
print(final['Text'].values[0])
print("*******************************")
print(list_of_sent[0])

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
*******************************
['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'were', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'hes', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is', 'in', 'college']


In [49]:
w2v_model = gensim.models.Word2Vec(list_of_sent, min_count=5, vector_size=50, workers=4)

In [53]:
words = list(w2v_model.wv.index_to_key)
print(len(words))

35274


In [54]:
w2v_model.wv.most_similar('tasty')

[('tastey', 0.8962493538856506),
 ('yummy', 0.8564350605010986),
 ('satisfying', 0.8378251791000366),
 ('filling', 0.8192634582519531),
 ('delicious', 0.8154632449150085),
 ('flavorful', 0.7947142124176025),
 ('tasteful', 0.7726016640663147),
 ('nutritious', 0.766100287437439),
 ('delish', 0.7537228465080261),
 ('addicting', 0.7520943880081177)]

In [55]:
w2v_model.wv.most_similar('like')

[('resemble', 0.7165138721466064),
 ('mean', 0.6540846824645996),
 ('prefer', 0.649625837802887),
 ('overpower', 0.6207175254821777),
 ('dislike', 0.6146439909934998),
 ('think', 0.6116567850112915),
 ('miss', 0.5870169997215271),
 ('overwhelm', 0.5854400396347046),
 ('enjoy', 0.5805496573448181),
 ('expect', 0.5753854513168335)]

In [66]:
count_vect_feat = count_vect.get_feature_names_out()
np.where(count_vect_feat == 'like')
print(count_vect_feat[1442686])

like


<b>Avg W2V, TFIDF-W2V</b>

In [67]:
#average Word2Vec
#compute average word2vec for each review

sent_vectors = []; #the avg-w2v for each sentence/review is stored in this
for sent in list_of_sent: #for each review/sentence
    sent_vec = np.zeros(50) #as word vectors are of zero length
    cnt_words = 0; #num of words with a valid vectors in the sentence/reviews
    for word in sent: #for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

364171
50


In [68]:
#TF-IDF weighted Word2vec
tfidf_feat = tf_idf_vect.get_feature_names_out() #tfid words/col-names

tfidf_sent_vectors = [];
row = 0;
for sent in list_of_sent:
    sent_vec = np.zeros(50)
    weight_sum = 0;
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

In [69]:
print(len(tfidf_sent_vectors))
print(len(tfidf_sent_vectors[0]))

364171
50
